<a href="https://colab.research.google.com/github/guruvittal/TimesFM/blob/main/TimesFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Libraries
packages = [
    ('einshape', 'einshape'),
    ('paxml', 'paxml'),
    ('praxis','praxis'),
    ('jax','jax'),
    ('fastapi','fastapi'),
    ('docker','docker'),
    ('utilsforecast','utilsforecast'),
    ('numpy','numpy'),
    ('pandas', 'pandas')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U --quiet --user



installing package einshape
installing package paxml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.8/686.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.5/772.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
#@title GCP Constants
PROJECT_ID = "argolis-project-340214"   # @param {type: "string"}
LOCATION = 'us-central1' # @param {type: "string"}
STAGING_BUCKET = 'firstargolisbucket' # @param {type: "string"}

In [2]:
#@title Auth
import sys

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

In [3]:
!pwd
from google.colab import files
files.view('/content')

/content


<IPython.core.display.Javascript object>

In [ ]:
!pip install git+https://github.com/google-research/timesfm

In [ ]:
!pip list -v


In [3]:
import json
import numpy as np
import pandas as pd
import logging

from google.cloud import aiplatform
from google.cloud.aiplatform.prediction import LocalModel
from google.cloud.aiplatform.docker_utils import build
from huggingface_hub.hf_api import HfFolder
from huggingface_hub import snapshot_download

import configparser
logging.basicConfig(level=logging.INFO)
aiplatform.init(project=PROJECT_ID,location=LOCATION, staging_bucket=STAGING_BUCKET)

config = configparser.ConfigParser()
config.read('config.ini')

[]

In [4]:

#@title Setup HuggingFace (TimesFM)

from google.colab import userdata
from huggingface_hub import HfFolder
from huggingface_hub import login

# You will need to change these variables
token = userdata.get('HF_TOKEN') #@param

login(token)

repo_id = "google/timesfm-1.0-200m"
folder = HfFolder()

# Authenticate with parameters


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# Add the structure to the file we will create
config.add_section('HuggingFace')
config.set('HuggingFace', 'hf_access_token', token)
with open(r"config.ini", 'w') as configfile:
    config.write(configfile)

In [6]:
HfFolder.save_token(config['HuggingFace']['hf_access_token'])

In [7]:
HF_TIMESFM_MODEL_NAME = "google/timesfm-1.0-200m"
GCS_TIMESFM_CKPT_PATH = f"{STAGING_BUCKET}/googleio24/models/timesfm/"

LOCAL_TIMESFM_CKPT_PATH = "models/timesfm/checkpoints_flax/"
PATH_TO_THE_SOURCE_DIR = "src/timesfm_serving"

PATH_TO_REQUIREMENTS_TXT = f"{PATH_TO_THE_SOURCE_DIR}/requirements.txt"

ARTIFACT_REPOSITORY = "googleio24"
TIMESFM_IMAGE = "timesfm-001-200m"

TIMESFM_IMAGE_URI = f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPOSITORY}/{TIMESFM_IMAGE}"
TIMESFM_MODEL_DISPLAY_NAME = "io24-timesfm-001-200m"

TIMESFM_ENDPOINT_DISPLAY_NAME = f"{TIMESFM_MODEL_DISPLAY_NAME}_endpoint"

BASE_IMAGE = "python:3.10"
DEFAULT_HTTP_PORT = 8501

DEFAULT_PREDICT_ROUTE = "/predict"
DEFAULT_HEALTH_ROUTE = "/health"

In [18]:
!echo $PATH_TO_THE_SOURCE_DIR

src/timesfm_serving


In [9]:
!mkdir -p $PATH_TO_THE_SOURCE_DIR
!touch $PATH_TO_THE_SOURCE_DIR/__init__.py

#Set project
!gcloud config set project $PROJECT_ID

# enable artifact registry service
!gcloud services enable artifactregistry.googleapis.com

# create docker repository in artifact registry
!gcloud artifacts repositories create $ARTIFACT_REPOSITORY \
    --repository-format=docker \
    --location=$LOCATION \
    --description="Artifact repository with TimesFM"

# configure auth for docker
!gcloud auth configure-docker $LOCATION-docker.pkg.dev --quiet

Updated property [core/project].
ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists
`docker` and `docker-credential-gcloud` need to be in the same PATH in order to work correctly together.
gcloud's Docker credential helper can be configured but it will not work until this is corrected.
Adding credentials for: us-central1-docker.pkg.dev
Docker configuration file updated.


In [19]:
print(GCS_TIMESFM_CKPT_PATH)

gs://firstargolisbucket/models/timesfm


In [12]:
# Sync the 'checkpoints' subdirectory to Google Cloud Storage (GCS)
GCS_TIMESFM_CKPT_PATH = f"gs://{STAGING_BUCKET}/models/timesfm"  # Replace with your GCS bucket


In [13]:
print("Getting HF model:", HF_TIMESFM_MODEL_NAME)
# Create a local directory to store the model checkpoint files
LOCAL_TIMESFM_CKPT_PATH="models/timesfm/checkpoints_flax"
!mkdir -p $LOCAL_TIMESFM_CKPT_PATH

# Download the model checkpoint files from the Hugging Face Hub
#HF_TIMESFM_MODEL_NAME = "google/timesfm-1.0-200m"  # Hugging Face model identifier
snapshot_download(
    repo_id=HF_TIMESFM_MODEL_NAME,
    local_dir=LOCAL_TIMESFM_CKPT_PATH,
    token=True  # Set to True if the model requires authentication
)

!gsutil -m cp -r $LOCAL_TIMESFM_CKPT_PATH/checkpoints $GCS_TIMESFM_CKPT_PATH

# Verify that the files were uploaded to GCS
!gsutil ls -r $GCS_TIMESFM_CKPT_PATH

Getting HF model: google/timesfm-1.0-200m


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Copying file://models/timesfm/checkpoints_flax/checkpoints/checkpoint_1100000/metadata/metadata [Content-Type=application/octet-stream]...
Copying file://models/timesfm/checkpoints_flax/checkpoints/checkpoint_1100000/descriptor/descriptor.pbtxt [Content-Type=application/octet-stream]...
Copying file://models/timesfm/checkpoints_flax/checkpoints/checkpoint_1100000/state/checkpoint [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, comp

In [20]:
#@title Create a Custom Predictor

%%writefile $PATH_TO_THE_SOURCE_DIR/predictor.py
"""Adapts a pretrained time-series foundation model TimesFM to the CPR framework.
Documentation for the model is here:
                           https://github.com/google-research/timesfm

Model checkpoints can be found here:
                           https://huggingface.co/google/timesfm-1.0-200m
"""

import os
from typing import Sequence, Any, Tuple, List, Dict, Union

from fastapi import HTTPException  # For handling API errors
from google.cloud.aiplatform.utils import prediction_utils  # Google Cloud AI Platform utilities

# from google.cloud.aiplatform import prediction as cpr  # Commented out, might be used later

import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations

from paxml import checkpoints  # For loading model checkpoints
import timesfm  # The TimesFM model library
import jax.numpy as jnp

# JAX Configuration (Accelerated Linear Algebra)
from jax._src import config
config.update("jax_platforms", "")  # Configures JAX to use available hardware accelerators (GPUs, TPUs)


# --- PREDICTION CLASS ---
# (Code likely continues from here, defining the predictor class and its methods)

class TimesFMPredictor:
    """Predictor class for Time Series Foundation Model (TimesFM)."""
    TIMESFM_MODEL_NAME = os.getenv("TIMESFM_MODEL_NAME", default="timesfm-1.0-200m")
    CONTEXT_LEN = 512
    HORIZON_LEN = 128
    INPUT_PATCH_LEN = 32
    OUTPUT_PATCH_LEN = 128
    NUM_LAYERS = 20
    MODEL_DIMS = 1280
    BACKEND = "cpu"  # or "gpu" if available

    def __init__(self):
        """Initialize the predictor class."""
        pass

    def load(self, artifacts_uri: str = ""):
        """Initializes the model and preprocessing transforms.

        Args:
            artifacts_uri: Directory where state dict is stored. Can be a
                GCS URI or local path.
        """
        if not (os.path.isdir(artifacts_uri) or artifacts_uri.startswith("gs://")):
            raise ValueError("Provided artifact_uri is not a directory.")

        print(f"Downloading checkpoints from {artifacts_uri}")
        prediction_utils.download_model_artifacts(artifacts_uri)

        artifact_path = os.getcwd()  # Assuming model is downloaded to current working directory


        print(f"Loading checkpoints from {artifact_path}")

        self._model = timesfm.TimesFm(
            context_len=self.CONTEXT_LEN,
            horizon_len=self.HORIZON_LEN,
            input_patch_len=self.INPUT_PATCH_LEN,
            output_patch_len=self.OUTPUT_PATCH_LEN,
            num_layers=self.NUM_LAYERS,
            model_dims=self.MODEL_DIMS,
            backend=self.BACKEND,
        )

        self._model.load_from_checkpoint(artifact_path, checkpoint_type=checkpoints.CheckpointType.FLAX)
        print(f"Loaded TimesFM model from {artifact_path}")

    def preprocess(self, request_dict: Dict[str, Dict]) -> Sequence[Any]:
          """Performs preprocessing.

          By default, the server expects a request body consisting of a valid JSON
          object. This will be parsed by the handler before it's evaluated by the
          preprocess method.

          Args:
              request_dict: Parsed request body. We expect that the input consists of
                  a list of time-series forecast contexts. Each context should be in
                  a format convertible to JTensor by `jnp.array`.

          Returns:
              Time-series forecast contexts are passed as is from the input as a list.
          """

          format = """
          {
              "instances": [
                  {
                      "inputs": [
                          [...],
                          [...]
                      ],
                      "freq": [
                          "0/1/2",
                          "0/1/2"
                      ]
                  }
              ]
          }
          """


          # Check for 'instances' key at the top level
          if "instances" not in request_dict:
              raise HTTPException(status_code=400, detail='Request must contain "instances" as a top-level key.')

          result = request_dict["instances"]
          # Check if 'instances' is a sequence (list or tuple) and is not empty
          if  isinstance(result, Sequence) and len(result) == 0:  # Simplified this condition
              raise HTTPException(
                  status_code=400,
                  detail=f"Invalid payload. Expected format is {format}. Received {type(result)}"
              )
          print("Result is:", result)

          result = result[0]

          print("Result[0] is:", result)


          if "inputs" in result:
              inputs = result["inputs"]  # Extract 'inputs'
              print("inputs is:", inputs)
              # Check if 'inputs' is a list or tuple, and each element is also a list or tuple.
              if not isinstance(inputs, Sequence) or not all(isinstance(x, Sequence) for x in inputs):
                  raise HTTPException(
                      status_code=400,
                      detail=f"Invalid datatype. Expected a list of time series forecast contexts. Received {type(inputs)}"
                  )
          else:
              raise HTTPException(
                  status_code=400,
                  detail=f"Invalid payload. Expected format is {format} with keys 'inputs' and 'freq' (frequencies)."
                  f"Received payload with type:{type(result)}",
              )
          print("Result is:", result)
          return result




    def predict(self, instances: Dict[str, List]) -> Tuple[List, List]:
        """Performs time series forecasting using the TimesFM model.

        Args:
            instances: A dictionary with two keys:
                - "inputs": List of time series forecast contexts. Each context should be convertible to a JAX NumPy array (jnp.array).
                - "freq": (Optional) Frequencies of each forecast context with values 0 (high), 1 (medium), or 2 (low).

        Returns:
            A tuple containing two lists:
                - point_forecast: Mean forecast of size (number of inputs, forecast horizon).
                - quantile_forecast: Full forecast (mean + quantiles) of size (number of inputs, forecast horizon, 1 + number of quantiles).

        Raises:
            ValueError: If the "inputs" key is missing in the input dictionary.
        """

        # Extract inputs
        inputs = instances["inputs"]

        """
        if inputs is None:
            raise ValueError("Missing 'inputs' key in the request data.")

        # Convert inputs to JAX arrays for efficient computation
        inputs = [jnp.array(ts_input) for ts_input in inputs]
        """

        # Extract frequencies (if provided) or use default (None)
        freq = None
        if "freq" in instances:
          freq = instances["freq"]

        # Generate forecasts using the TimesFM model
        point_forecast, quantile_forecast = self._model.forecast(inputs, freq)
        # Convert forecasts to standard Python lists for serialization
        point_forecast = point_forecast.tolist()
        quantile_forecast = quantile_forecast.tolist()

        return point_forecast, quantile_forecast

    def postprocess(self, forecasts: Tuple[List, List]) -> Dict[str, List[Dict[str, Union[int, float]]]]:
        """Translate the model output

        Args:
            forecasts: A tuple of List
                - the mean forecast of size (# inputs, # forecast horizon),
                - the full forecast (mean + quantiles) of size
                    (#inputs, # forecast horizon, 1 + # quantiles).

        Returns:
            Dictionary containing the list of point forecasts and quantile forecasts
            for each of the input time-series context.
        """
        point_forecasts, quantile_forecasts = forecasts
        predictions = [
            dict(
                point_forecast=points,
                quantile_forecast=quantiles,
            )
            for (points, quantiles) in zip(point_forecasts, quantile_forecasts)
        ]

        print("Quantile forecasts:", quantile_forecasts)

        return {"predictions": predictions}


Overwriting src/timesfm_serving/predictor.py


In [15]:
!git clone https://github.com/google-research/timesfm

Cloning into 'timesfm'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 121 (delta 55), reused 102 (delta 46), pack-reused 1
Receiving objects: 100% (121/121), 547.21 KiB | 12.73 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [21]:
!echo $GCS_TIMESFM_CKPT_PATH
print("HF Model:", HF_TIMESFM_MODEL_NAME)

gs://firstargolisbucket/models/timesfm
HF Model: google/timesfm-1.0-200m


In [23]:
#@title Simple Unit test with Predictor

# 4. Run simple unit test to ensure predictor works [Optional]
from praxis.layers import transformers
import sys

import timesfm
from src.timesfm_serving.predictor import TimesFMPredictor

# define predictor
predictor = TimesFMPredictor()

# Load model checkpoint
predictor.load(f"{GCS_TIMESFM_CKPT_PATH}/checkpoints")

# create payload
inputs = [list(np.linspace(0, 1, 100)), list(np.sin(np.linspace(0, 20, 200)))]
payload = {"instances": [{"inputs": inputs}]}

print(payload)

Loading checkpoints from /content
Multiprocessing context has already been set.
Constructing model weights.


Constructed model weights in 7.99 seconds.
Restoring checkpoint from /content.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 5.49 seconds.
Jitting decoding.
Jitted decoding in 60.58 seconds.
Loaded TimesFM model from /content
{'instances': [{'inputs': [[0.0, 0.010101010101010102, 0.020202020202020204, 0.030303030303030304, 0.04040404040404041, 0.05050505050505051, 0.06060606060606061, 0.07070707070707072, 0.08080808080808081, 0.09090909090909091, 0.10101010101010102, 0.11111111111111112, 0.12121212121212122, 0.13131313131313133, 0.14141414141414144, 0.15151515151515152, 0.16161616161616163, 0.17171717171717174, 0.18181818181818182, 0.19191919191919193, 0.20202020202020204, 0.21212121212121213, 0.22222222222222224, 0.23232323232323235, 0.24242424242424243, 0.25252525252525254, 0.26262626262626265, 0.27272727272727276, 0.2828282828282829, 0.29292929292929293, 0.30303030303030304, 0.31313131313131315, 0.32323232323232326, 0.33333333333333337, 0.3434343434343435, 0.3535353535353536, 0.36363636363636365, 0.37373737373737376, 0.38383838383838387, 0.393939393939394, 0.4040404040404041, 0.4141

In [24]:
#@title Preprocess Payload, Predict & Generate output

# preprocess, predict and post-process
preprocessed_inputs = predictor.preprocess(payload)
print("Preprocessed inputs:", preprocessed_inputs)
pt_forecast, quantil_forecast = predictor.predict(preprocessed_inputs)
print("Predict output:", pt_forecast, quantil_forecast)
result = pt_forecast, quantil_forecast
preprocessed_outputs = predictor.postprocess(result)
print(preprocessed_outputs)


Result is: [{'inputs': [[0.0, 0.010101010101010102, 0.020202020202020204, 0.030303030303030304, 0.04040404040404041, 0.05050505050505051, 0.06060606060606061, 0.07070707070707072, 0.08080808080808081, 0.09090909090909091, 0.10101010101010102, 0.11111111111111112, 0.12121212121212122, 0.13131313131313133, 0.14141414141414144, 0.15151515151515152, 0.16161616161616163, 0.17171717171717174, 0.18181818181818182, 0.19191919191919193, 0.20202020202020204, 0.21212121212121213, 0.22222222222222224, 0.23232323232323235, 0.24242424242424243, 0.25252525252525254, 0.26262626262626265, 0.27272727272727276, 0.2828282828282829, 0.29292929292929293, 0.30303030303030304, 0.31313131313131315, 0.32323232323232326, 0.33333333333333337, 0.3434343434343435, 0.3535353535353536, 0.36363636363636365, 0.37373737373737376, 0.38383838383838387, 0.393939393939394, 0.4040404040404041, 0.4141414141414142, 0.42424242424242425, 0.43434343434343436, 0.4444444444444445, 0.4545454545454546, 0.4646464646464647, 0.474747474

In [25]:
%%writefile $PATH_TO_THE_SOURCE_DIR/main.py
import os

import json
import flask

from predictor import TimesFMPredictor

# Creation of the Flask app
app = flask.Flask(__name__)

# define predictor and Load model checkpoint
predictor = TimesFMPredictor()

predictor.load(os.environ['AIP_STORAGE_URI'])

@app.route(os.environ['AIP_HEALTH_ROUTE'], methods=['GET'])
def health() -> flask.Response:
    return flask.Response(status=200)

@app.route(os.environ['AIP_PREDICT_ROUTE'], methods=['GET', 'POST'])
def predict() -> flask.Response:
    try:
        body = flask.request.get_json(silent=True, force=True)
        preprocessed_inputs = predictor.preprocess(body)
        outputs = predictor.predict(preprocessed_inputs)
        postprocessed_outputs = predictor.postprocess(outputs)
        return flask.Response(json.dumps(postprocessed_outputs), status=200, mimetype='application/json')

        except Exception as e:  # pylint: disable=broad-except-caught
            return flask.Response(json.dumps({'error': str(e)}), status=500, mimetype='application/json')

        if __name__ == '__main__':
            app.run(host='0.0.0.0', port=os.environ['AIP_HTTP_PORT'])

Writing src/timesfm_serving/main.py


In [42]:
#@title Create Requirements.txt file in timesfm_serving
%%writefile src/timesfm_serving/requirements.txt
einshape
paxml
praxis
jax
numpy
pandas
utilsforecast
fastapi
docker

Writing src/timesfm_serving/requirements.txt


In [45]:
#@title Step 5: Build a Custom Container
%%writefile src/Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.10

# install dependencies
COPY timesfm_serving/requirements.txt /app/

RUN pip install -r /app/requirements.txt

# copy app files
COPY timesfm_serving/ /app

# download timesfm inference modules
RUN git clone https://github.com/google-research/timesfm.git

RUN rm timesfm/__init__.py
RUN sed -i 's#^from src#from .#g' timesfm/src/timesfm.py

# run inference
CMD ["python", "main.py"]


Overwriting src/Dockerfile


In [38]:
!echo $TIMESFM_IMAGE_URI

us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m


In [46]:
%cd src
!gcloud builds submit --tag=$TIMESFM_IMAGE_URI
%cd ..

/content/src
Creating temporary archive of 7 file(s) totalling 16.4 KiB before compression.
Uploading tarball of [.] to [gs://argolis-project-340214_cloudbuild/source/1716513794.648469-5b9660f810954df8aa352c2e483d6291.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/argolis-project-340214/locations/global/builds/0e3254ec-7ca6-4c6c-9d4b-e6eb1727cb63].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/0e3254ec-7ca6-4c6c-9d4b-e6eb1727cb63?project=742458474659 ].
Waiting for build to complete. Polling interval: 1 second(s).
 REMOTE BUILD OUTPUT
starting build "0e3254ec-7ca6-4c6c-9d4b-e6eb1727cb63"

FETCHSOURCE
Fetching storage object: gs://argolis-project-340214_cloudbuild/source/1716513794.648469-5b9660f810954df8aa352c2e483d6291.tgz#1716513795062969
Copying gs://argolis-project-340214_cloudbuild/source/1716513794.648469-5b9660f810954df8aa352c2e483d6291.tgz#1716513795062969...
/ [1 files][  6.0 KiB/  6.0 KiB]                                             

In [26]:
!echo $TIMESFM_IMAGE_URI

us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m


In [27]:
DEPLOY_IMAGE = "https://cloudbuild.googleapis.com/v1/projects/argolis-project-340214/locations/global/builds/0e3254ec-7ca6-4c6c-9d4b-e6eb1727cb63"

In [80]:
#@title Run the container locally
local_model = LocalModel(
    serving_container_image_uri=TIMESFM_IMAGE_URI,
    serving_container_predict_route=DEFAULT_PREDICT_ROUTE,
    serving_container_health_route=DEFAULT_HEALTH_ROUTE,
    serving_container_ports=[DEFAULT_HTTP_PORT],
)

In [81]:
local_model.get_serving_container_spec()

image_uri: "us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m"
ports {
  container_port: 8501
}
predict_route: "/predict"
health_route: "/health"

In [82]:
inputs = [list(np.linspace(0, 1, 100)), list(np.sin(np.linspace(0, 20, 200)))]
payload = {"instances": [{"inputs": inputs}]}

with open('payload.json', 'w') as f:
    json.dump(payload, f)

In [90]:
!sudo service docker restart

In [61]:
from google.cloud import aiplatform
print("aiplatform version: ", aiplatform.__version__)

aiplatform version:  1.52.0


In [ ]:
IS_COLAB = False
if "google.colab" in sys.modules:
  IS_COLAB = True
#%%bash -s $IS_COLAB $DEPLOY_IMAGE $TIMESFM_IMAGE_URI


In [41]:
TIMESFM_IMAGE_URI

'us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m'

In [71]:
!TIMESFM_IMAGE_URI=$TIMESFM_IMAGE_URI
!DEPLOY_IMAGE=$DEPLOY_IMAGE


In [83]:
%%bash -s $TIMESFM_IMAGE_URI $DEPLOY_IMAGE
set -x
echo "Got param1 as:" $1
echo "Got param2 as:" $2
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $1
docker tag $1 $2
docker push $2
kill $(jobs -p)

Got param1 as: us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m
Got param2 as: https://cloudbuild.googleapis.com/v1/projects/argolis-project-340214/locations/global/builds/0e3254ec-7ca6-4c6c-9d4b-e6eb1727cb63
Using default tag: latest
latest: Pulling from argolis-project-340214/googleio24/timesfm-001-200m
c6cf28de8a06: Already exists
891494355808: Already exists
6582c62583ef: Already exists
bf2c3e352f3d: Already exists
a99509a32390: Already exists
946285778af4: Already exists
b2ab5d29389b: Already exists
d76e704b1be9: Already exists
32b14c58fcfc: Already exists
49c9ff2efd6b: Already exists
5119e73f1536: Already exists
b82c8e4a85c6: Already exists
30c0e033ca4b: Already exists
88ccd20b36d3: Already exists
abf00b58c1b0: Pulling fs layer
8e6b754449a4: Pulling fs layer
4f4fb700ef54: Pulling fs layer
12eeb765d0fa: Pulling fs layer
9bab74df5c6b: Pulling fs layer
e64e5e7eb223: Pulling fs layer
e4f838478090: Pulling fs layer
74e15fee10a2: Pulling fs layer
af22d1c1dd

+ echo 'Got param1 as:' us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m
+ echo 'Got param2 as:' https://cloudbuild.googleapis.com/v1/projects/argolis-project-340214/locations/global/builds/0e3254ec-7ca6-4c6c-9d4b-e6eb1727cb63
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
+ dockerd -b none --iptables=0 -l warn
time="2024-05-24T02:28:21.838829849Z" level=warning msg="failed to load plugin io.containerd.snapshotter.v1.devmapper" error="devmapper not configured"
time="2024-05-24T02:28:21.839344540Z" level=warning msg="could not use snapshotter devmapper in metadata plugin" error="devmapper not configured"
time="2024-05-24T02:28:21.841938820Z" level=error msg="failed to initialize a tracing processor \"otlp\"" error="no OpenTelemetry endpoint: skip plugin"
time="2024-05-24T02:28:21.887473307Z" level=error msg="failed to mount overlay: invalid argument" storage-driver=overlay2
time="2024-05-24T02:28:21.887602376Z" level=error msg="

In [99]:
%%writefile dockerstart.sh
nohup dockerd -b none --iptables=0 -l warn &
#cat /var/run/docker.sock

Overwriting dockerstart.sh


In [102]:
%%bash
chmod +x dockerstart.sh
./dockerstart.sh

Process is interrupted.


In [94]:
%%bash
ps -ef | grep docker

root           1       0  0 15:49 ?        00:00:01 /sbin/docker-init -- /datalab/run.sh
root      113976  113974  0 23:22 ?        00:00:00 grep docker


In [47]:

%%bash -s $TIMESFM_IMAGE_URI $DEPLOY_IMAGE
set -x
echo "Got param1 as:" $1
echo "Got param2 as:" $2
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $1


Process is interrupted.


In [88]:
endpoint = local_model.deploy_to_local_endpoint(    artifact_uri=f'{GCS_TIMESFM_CKPT_PATH}/checkpoints',
    host_port=DEFAULT_HTTP_PORT)

In [91]:
!ls "/var/run/docker.sock"

ls: cannot access '/var/run/docker.sock': No such file or directory


In [ ]:
#endpoint.artifact_uri = f'{GCS_TIMESFM_CKPT_PATH}/checkpoints'
print("Serve:", endpoint.serve())
print(endpoint.artifact_uri,
endpoint.container_port,
endpoint.container_is_running,
      endpoint.run_health_check())

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f'{GCS_TIMESFM_CKPT_PATH}/checkpoints',
    host_port=DEFAULT_HTTP_PORT
) as local_endpoint:

    health_check_response = local_endpoint.run_health_check()
    predict_response = local_endpoint.predict(
        request_file='payload.json',
        headers={"Content-Type": "application/json"}
    )

In [45]:
%%shell

sudo apt update -qq

sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable" &

sudo apt update -qq

sudo apt install docker-ce


45 packages can be upgraded. Run 'apt list --upgradable' to see them.
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.16).
software-properties-common is already the newest version (0.99.22.9).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,510 B of archives.
After this operation, 170 kB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package apt-tran

In [ ]:
!sudo dockerd
#!systemctl status docker
#!sudo systemctl start docker
#!docker build -t my-image .

In [31]:
#%%shell

!TIMESFM_IMAGE_URI=$TIMESFM_IMAGE_URI
!echo $TIMESFM_IMAGE_URI
#!dockerd -b none --iptables=0 -l warn &
!dockerd -b none --iptables=0 -l warn &
!docker  build --tag=$TIMESFM_IMAGE_URI ./src/

us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m
WARN[2024-05-24T00:19:56.633298637Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2024-05-24T00:19:56.634507569Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2024-05-24T00:19:56.636233559Z] failed to initialize a tracing processor "otlp"  error="no OpenTelemetry endpoint: skip plugin"
ERRO[2024-05-24T00:19:56.673711133Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2024-05-24T00:19:56.673884998Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
WARN[2024-05-24T00:19:56.715004047Z] Failed to disable IPv6 on all interfaces on network namespace "/var/run/docker/netns/65678b632bca": failed to disable IPv6 forwarding for container's interface all: open /proc/sys/net/ipv6/conf/all/disable_ipv6: read-only file system 
ERRO[2024-05-24T00:19:56.

In [92]:
# Check version.
!docker --version

Docker version 24.0.2, build cb74dfc


In [ ]:
!docker  build --tag=$TIMESFM_IMAGE_URI ./src/

In [26]:
%%shell
pip install udocker
udocker --allow-root install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 1.2 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.16
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful


In [ ]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker pull hello-world
docker images
docker run hello-world
kill $(jobs -p)

In [103]:
#@title Run on Vertex AI Using Vertex AI Model Registry

local_model.push_image()

/usr/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/usr/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


DockerError: ('\nDocker failed with error code 1.\nCommand: docker push us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m\n', ['docker', 'push', 'us-central1-docker.pkg.dev/argolis-project-340214/googleio24/timesfm-001-200m'], 1)